# Mirrored Strategy: Basic

## Imports

In [2]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.8 MB/s eta 0:00:0000:01
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [3]:
# Import TensorFlow and TensorFlow Datasets

import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import os

2024-07-22 13:56:13.816405: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 13:56:13.816532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 13:56:13.938663: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Load the dataset we'll use for this lab
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True, data_dir='./data')

mnist_train, mnist_test = datasets['train'], datasets['test']

Dataset mnist downloaded and prepared to ./data/mnist/3.0.1. Subsequent calls will reuse this data.


In [5]:
# Define the strategy to use and print the number of devices found
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


In [12]:
# Get the number of examples in the train and test sets
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
# Use for Mirrored Strategy
# BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
# Use for No Strategy
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * 1

In [13]:
# Function for normalizing the image
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

In [16]:
# Set up the train and eval data set
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [17]:
# Use for Mirrored Strategy -- comment out `with strategy.scope():` and deindent for no strategy
# with strategy.scope():
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=tf.keras.optimizers.Adam(),
            metrics=['accuracy'])    

In [18]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [19]:
model.fit(train_dataset, epochs=12)

Epoch 1/12
938/938 [==============================] - 4s 3ms/step - loss: 0.2061 - accuracy: 0.9406
Epoch 2/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0673 - accuracy: 0.9798
Epoch 3/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0468 - accuracy: 0.9860
Epoch 4/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0347 - accuracy: 0.9895
Epoch 5/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0256 - accuracy: 0.9924
Epoch 6/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0196 - accuracy: 0.9938
Epoch 7/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0157 - accuracy: 0.9954
Epoch 8/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0125 - accuracy: 0.9958
Epoch 9/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0091 - accuracy: 0.9975
Epoch 10/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0072 - accuracy: 0.9977

In [ ]:
Epoch 1/12
WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1721656590.036209     116 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
938/938 [==============================] - 8s 5ms/step - loss: 0.2093 - accuracy: 0.9388
Epoch 2/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0710 - accuracy: 0.9792
Epoch 3/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0494 - accuracy: 0.9849
Epoch 4/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0371 - accuracy: 0.9887
Epoch 5/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0280 - accuracy: 0.9916
Epoch 6/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0213 - accuracy: 0.9934
Epoch 7/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0162 - accuracy: 0.9951
Epoch 8/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0128 - accuracy: 0.9963
Epoch 9/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0102 - accuracy: 0.9968
Epoch 10/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0078 - accuracy: 0.9977
Epoch 11/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0076 - accuracy: 0.9974
Epoch 12/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0054 - accuracy: 0.9984
<keras.src.callbacks.History at 0x7f6989aab070>